In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import h5py
from pathlib import Path

import pandas as pd
import numpy as np
import flammkuchen as fl

from tqdm import tqdm

In [3]:
# from dlc_prepocess_helper import *

In [4]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

def are_points_in_circle(points, a, b, r):
    # calculate the distance from each point to the center of the circle
    distances = (points[:, 0] - a)**2 + (points[:, 1] - b)**2

    # check if the distances are less than or equal to the square of the radius
    return distances <= r**2

def resample(df, df2):
    #df gets resampled to df2
    df1 =df.copy()
    resampled_df = pd.DataFrame(np.zeros((df2.shape[0], df1.shape[1])), columns=df1.columns)

    for i in df1.columns:
        interpolated_data = np.interp(df2.t, df1.t, df1[i])
        resampled_df[i] = interpolated_data

    return resampled_df

def exptrapolate_segments(tail_x, tail_y, N_seg):
    T = tail_x.shape[1]
    tail_x_10 = np.zeros((N_seg+1,T))
    tail_y_10 = np.zeros((N_seg+1,T))

    for i in range(T):
        points = np.array([tail_x[:,i],tail_y[:,i]]).T  # a (nbre_points x nbre_dim) array

        alpha = np.linspace(0, 1, 11)
        # Linear length along the line:
        distance = np.cumsum( np.sqrt(np.sum( np.diff(points, axis=0)**2, axis=1 )) )
        distance = np.insert(distance, 0, 0)/distance[-1]
        interpolator =  interp1d(distance, points, kind='cubic', axis=0)
        curve = interpolator(alpha)
        tail_x_10[:,i] = curve[:,0]
        tail_y_10[:,i] = curve[:,1]
    return tail_x_10, tail_y_10

def midpoint(x1, y1, x2, y2):
    return ((x1 + x2)/2, (y1 + y2)/2)

def mid_head(right_eye_posterior_x, right_eye_posterior_y,
             right_eye_anterior_x, right_eye_anterior_y,
             left_eye_posterior_x, left_eye_posterior_y,
             left_eye_anterior_x, left_eye_anterior_y
):
    left_mid_eye_y = (left_eye_anterior_y+left_eye_posterior_y)/2
    left_mid_eye_x = (left_eye_anterior_x+left_eye_posterior_x)/2

    right_mid_eye_x = (right_eye_anterior_x+right_eye_posterior_x)/2
    right_mid_eye_y = (right_eye_anterior_y+right_eye_posterior_y)/2

    mid_headx, mid_heady = midpoint(left_mid_eye_x,left_mid_eye_y, right_mid_eye_x, right_mid_eye_y) #xy left, xy right

    return mid_headx, mid_heady, left_mid_eye_x, left_mid_eye_y, right_mid_eye_x, right_mid_eye_y

def get_eye_points(eye_coords):
    # (18900, 1, 2, 2, 2) or (18900, 2, 2, 2)
    left_eye_points = np.asarray(eye_coords)[:,0]
    right_eye_points = np.asarray(eye_coords)[:,1]
    
    l_anterior = np.asarray(left_eye_points[:,0])
    l_posterior = np.asarray(left_eye_points[:,1])
    
    left_eye_anterior_x = []
    left_eye_anterior_y = []
    left_eye_posterior_x = []
    left_eye_posterior_y = []
    for i in range(l_anterior.shape[0]):
        left_eye_anterior_x.append(l_anterior[i][0])
        left_eye_anterior_y.append(l_anterior[i][1])
        left_eye_posterior_x.append(l_posterior[i][0])
        left_eye_posterior_y.append(l_posterior[i][1])
        
    r_anterior = np.asarray(right_eye_points[:,0])
    r_posterior = np.asarray(right_eye_points[:,1])
    
    right_eye_anterior_x = []
    right_eye_anterior_y = []
    right_eye_posterior_x = []
    right_eye_posterior_y = []
    for i in range(r_anterior.shape[0]):
        right_eye_anterior_x.append(r_anterior[i][0])
        right_eye_anterior_y.append(r_anterior[i][1])
        right_eye_posterior_x.append(r_posterior[i][0])
        right_eye_posterior_y.append(r_posterior[i][1])

    return  left_eye_anterior_x, left_eye_anterior_y, left_eye_posterior_x, left_eye_posterior_y, right_eye_anterior_x, right_eye_anterior_y, right_eye_posterior_x, right_eye_posterior_y,


def compute_angle_between_vect_tail(v1, v2):
    dot = np.einsum('ijk,ijk->ij',[v1,v1,v2],[v2,v1,v2])
    cos_= dot[0,:]
    sin_= np.cross(v1,v2)
    angle_= np.arctan2(sin_,cos_)
    return angle_


def compute_angle_between_vect(u,v):
    u = u/np.linalg.norm(u)
    v = v/np.linalg.norm(v)
    return np.arctan2(u[0]*v[1]-u[1]*v[0],u[0]*v[0]+u[1]*v[1])

def compute_body_angle(head_x, head_y, body_x, body_y):
    """
    Computes the angle between two points in 2D space.
    
    Parameters:
    head_x, head_y: Coordinates of the first point (head).
    body_x, body_y: Coordinates of the second point (body).
    
    Returns:
    angles_radians: The angle in radians.
    angles_degrees: The angle in degrees.
    """
    # Calculate the differences in the x and y coordinates
    delta_x = body_x - head_x
    delta_y = body_y - head_y

    # Calculate the angle using numpy's arctan2
    angles_radians = np.arctan2(delta_y, delta_x)

    # Convert the angle from radians to degrees
    angles_degrees = np.degrees(angles_radians)

    return angles_radians, angles_degrees


### compute fin and body angles
def fin_preprocess(df, mid_headx, mid_heady, body_x, body_y):
    ##Fin angle computatright
    #Fin angle computatright
    right_fin_tip_x =  df['right_fin_tip'].values[:, 0].astype('float')
    right_fin_tip_y =  df['right_fin_tip'].values[:, 1].astype('float')
    right_fin_base_x =  df['right_fin_base'].values[:, 0].astype('float')
    right_fin_base_y =  df['right_fin_base'].values[:, 1].astype('float')

    left_fin_tip_x =  df['left_fin_tip'].values[:, 0].astype('float')
    left_fin_tip_y =  df['left_fin_tip'].values[:, 1].astype('float')
    left_fin_base_x =   df['left_fin_base'].values[:, 0].astype('float')
    left_fin_base_y =   df['left_fin_base'].values[:, 1].astype('float')

    # lets make all the vectors
    a = left_fin_base_x-left_fin_tip_x
    b = left_fin_base_y-left_fin_tip_y
    left_fin_vect = np.array([b,-a])

    a = right_fin_base_x-right_fin_tip_x 
    b = right_fin_base_y-right_fin_tip_y
    right_fin_vect = np.array([-b,a])

    body_vect = np.vstack((mid_headx -body_x , mid_heady - body_y)) 

    ## Compute angles between vectors
    left_fin_angle =  compute_angle_between_vect(left_fin_vect, body_vect)
    right_fin_angle =  compute_angle_between_vect(right_fin_vect, body_vect)

    #nan movement artifacts
    left_fin_angle = left_fin_angle - left_fin_angle[0]
    right_fin_angle = right_fin_angle - right_fin_angle[0]
    left_fin_angle[abs(np.diff(left_fin_angle, prepend=[0])) >= 2] = 0 #np.nan #np.pi
    right_fin_angle[abs(np.diff(right_fin_angle, prepend=[0])) >= 2] = 0 #np.nan #np.pi
    
    return left_fin_vect, right_fin_vect, left_fin_angle, right_fin_angle

# Load Bouts

In [5]:
# master_path = Path(r'\\portulab.synology.me\data\Kata\testdata\Raw_Data')

# fish_paths = list(master_path.glob('*f[0-9]*'))
# fish_paths, len(fish_paths)


In [6]:
## Analysed for paper

# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\230307_visstim_2D") #rectangular arena # start from fish 1
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\22042024_visstim_2D_round")
master_path = Path(r"\\portulab.synology.me\data\Kata\Data\22042024_visstim_2D_2") #rectangular arena
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\13052024_visstim_2D_round")
# master_path = Path(r"\\portulab.synology.me\data\Kata\Data\14052024_visstim_2D_round")

In [7]:
fish_paths = list(master_path.glob('*f[0-9]*'))
fish_paths, len(fish_paths)


([WindowsPath('//portulab.synology.me/data/Kata/Data/22042024_visstim_2D_2/240422_f0'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/22042024_visstim_2D_2/240422_f0_1'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/22042024_visstim_2D_2/240422_f0_2'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/22042024_visstim_2D_2/240422_f1'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/22042024_visstim_2D_2/240422_f1_1'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/22042024_visstim_2D_2/240422_f2'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/22042024_visstim_2D_2/240422_f3'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/22042024_visstim_2D_2/240422_f4'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/22042024_visstim_2D_2/240422_f5'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/22042024_visstim_2D_2/240422_f6'),
  WindowsPath('//portulab.synology.me/data/Kata/Data/22042024_visstim_2D_2/240422_f7'),
  WindowsPath('//portulab.

In [8]:
fish= 3
fish_path = fish_paths[fish]
fish_id =  fish_paths[fish].name#[:-13]
exp_name = Path(fish_paths[fish]).parts[-2]
# exp_name = 'testfish'
exp_name, fish_id

('22042024_visstim_2D_2', '240422_f1')

In [9]:
# out_path = Path(r'\\portulab.synology.me\data\Kata\testdata\Processed_Data')

In [10]:
## Analysed for paper

# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\230307_visstim_2D_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\22042024_visstim_2D_round_")
out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\22042024_visstim_2D_2_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\13052024_visstim_2D_round_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\Processed_Data\14052024_visstim_2D_round_")


In [11]:
round_arena= False #True
thresh = 0.99

In [12]:
fps=200
mm_per_unit = 1/70
N_seg = 10

### Loop

In [ ]:

for ind, fish_path in enumerate(tqdm(fish_paths)):
    fish_id =  fish_path.name
    print ('Working on fish {}'.format(fish_id))

    filename = list(fish_path.glob('*316000.h5*'))[0]
    df= pd.read_hdf(filename,  header=[1, 2], index_col=0)
    df = df['DLC_resnet50_dlc_2Dec12shuffle1_316000']
    print(f'{df.shape[0]/(fps*60)} minutes at {fps} fps')
    print('working on {} frames'.format(df.shape[0]))
    
    #Extract angles
    body_x = df.body.values[:, 0].astype('float')
    body_y = df.body.values[:, 1].astype('float')
    
    tail_x_col = [f'tail_{i}' for i in range(5)]
    tail_y_col = [f'tail_{i}' for i in range(5)]
    tail_x = np.array([df[x].iloc[:, 0].values.astype('float') for x in tail_x_col])
    tail_y = np.array([df[x].iloc[:, 1].values.astype('float') for x in tail_y_col])
    
    # upsample tail to 10 segments
    tail_x_10, tail_y_10 = exptrapolate_segments(tail_x, tail_y, N_seg)

    eye_coords = fl.load(fish_path/'eye_coords.h5')['eye_coords']
    print (eye_coords.shape)
    left_eye_anterior_x, left_eye_anterior_y, left_eye_posterior_x, left_eye_posterior_y, right_eye_anterior_x, right_eye_anterior_y, right_eye_posterior_x, right_eye_posterior_y = get_eye_points(eye_coords)

    mid_headx, mid_heady, left_mid_eye_x, left_mid_eye_y, right_mid_eye_x, right_mid_eye_y = mid_head(
             np.asarray(right_eye_posterior_x), np.asarray(right_eye_posterior_y),
             np.asarray(right_eye_anterior_x), np.asarray(right_eye_anterior_y),
             np.asarray(left_eye_posterior_x), np.asarray(left_eye_posterior_y),
             np.asarray(left_eye_anterior_x), np.asarray(left_eye_anterior_y))

    # Now, let's add 'head_x' and 'head_y' keypoints under 'head' with the same likelihood as 'body'
    for i in range(11):
        df[('tail_{}'.format(i), 'x')] = tail_x_10[i,:] 
        df[('tail_{}'.format(i), 'y')] = tail_y_10[i,:]
        df[('tail_{}'.format(i), 'likelihood')] = df[('body', 'likelihood')] # Copy likelihood of body to head

    df[('left_eye_anterior', 'x')] = left_eye_anterior_x 
    df[('left_eye_anterior', 'y')] = left_eye_anterior_y 
    
    df[('left_eye_posterior', 'x')] = left_eye_posterior_x 
    df[('left_eye_posterior', 'y')] = left_eye_posterior_y 
    
    df[('right_eye_anterior', 'x')] = right_eye_anterior_x 
    df[('right_eye_anterior', 'y')] = right_eye_anterior_y 
    
    df[('right_eye_posterior', 'x')] = right_eye_posterior_x 
    df[('right_eye_posterior', 'y')] = right_eye_posterior_y 

    # Now, let's add 'head_x' and 'head_y' keypoints under 'head' with the same likelihood as 'body'
    df[('mid_head', 'x')] = mid_headx 
    df[('mid_head', 'y')] = mid_heady 
    df[('mid_head', 'likelihood')] = df[('body', 'likelihood')] # Copy likelihood of body to head

    motor_path = list(fish_path.glob('*motor_log.hdf5*'))[0]
    motor_log = pd.read_hdf(motor_path)
    tp = np.linspace(motor_log.t.iloc[0], motor_log.t.iloc[-1], df.shape[0])
    df = df.copy()
    df['t'] = tp
    resampled = resample(motor_log, df)
    motor_x = resampled.x.values.astype('float')
    motor_y = resampled.y.values.astype('float')
    motor_z = resampled.z.values.astype('float')
    df.insert(0, ("motor", "x"), motor_x.T)
    df.insert(0, ("motor", "y"), motor_y.T)
    df.insert(0, ("motor", "z"), motor_z.T)

    if round_arena:
        print ('round arena')
        mask  = fl.load(master_path / '{}_mask.csv'.format(exp_name))
        #for round marks
        margin = mask[-1]
        center_x = mask[0]
        center_y = mask[1]
        radius = mask[2] 
    
        points = np.column_stack((resampled.x,resampled.y))
        to_fill = are_points_in_circle(points, center_x, center_y, radius)
        df.insert(0, ("edge_filter", "ok"), to_fill)
    else:
        print ('rect arena')
        # mask  = fl.load(fish_path/'filter'/'mask.csv'.format(exp_name))
        mask  = fl.load(master_path/ 'mask.csv'.format(exp_name))
        # for rectangular mask
        margin = mask[4]
        x_lim_neg = mask[0]
        x_lim_pos = mask[1]
        y_lim_neg = mask[2]
        y_lim_pos = mask[3]
        
        filtered_x = resampled[(resampled.x >= x_lim_neg + margin ) & (resampled.x <= x_lim_pos - margin )]
        filtered = filtered_x[(resampled.y >= y_lim_neg + margin ) & (resampled.y <= y_lim_pos - margin )]
        indixes_ok = filtered.index.values
        to_fill = np.full(resampled.shape[0], False)
        to_fill[indixes_ok] = True
        df.insert(0, ("edge_filter", "ok"), to_fill)

    good_dlc_tracking = df[df.body.likelihood >=thresh].index.values
    to_fill = np.full(df.shape[0],-1)
    to_fill[good_dlc_tracking] = 1
    df.insert(0, ("DLC_filter", "pos"), to_fill)

    df.to_csv(out_path/ '{}_DLC_mod.csv'.format(fish_id))

  0%|                                                                                           | 0/13 [00:00<?, ?it/s]

Working on fish 240422_f0
9.354083333333334 minutes at 200 fps
working on 112249 frames
(112249, 2, 2, 2)
rect arena


C:\Users\admin\AppData\Local\Temp\ipykernel_14372\135503381.py:93: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = filtered_x[(resampled.y >= y_lim_neg + margin ) & (resampled.y <= y_lim_pos - margin )]
  8%|██████▍                                                                            | 1/13 [00:48<09:36, 48.05s/it]

Working on fish 240422_f0_1
9.003 minutes at 200 fps
working on 108036 frames
(108036, 2, 2, 2)
rect arena


C:\Users\admin\AppData\Local\Temp\ipykernel_14372\135503381.py:93: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = filtered_x[(resampled.y >= y_lim_neg + margin ) & (resampled.y <= y_lim_pos - margin )]
 15%|████████████▊                                                                      | 2/13 [01:33<08:34, 46.77s/it]

...

C:\Users\admin\AppData\Local\Temp\ipykernel_14372\135503381.py:93: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = filtered_x[(resampled.y >= y_lim_neg + margin ) & (resampled.y <= y_lim_pos - margin )]
 62%|███████████████████████████████████████████████████                                | 8/13 [05:57<03:40, 44.01s/it]

Working on fish 240422_f5
6.742416666666666 minutes at 200 fps
working on 80909 frames
(80909, 2, 2, 2)
rect arena


C:\Users\admin\AppData\Local\Temp\ipykernel_14372\135503381.py:93: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = filtered_x[(resampled.y >= y_lim_neg + margin ) & (resampled.y <= y_lim_pos - margin )]
 69%|█████████████████████████████████████████████████████████▍                         | 9/13 [06:31<02:43, 40.83s/it]

Working on fish 240422_f6
7.639666666666667 minutes at 200 fps
working on 91676 frames
(91676, 2, 2, 2)
rect arena


C:\Users\admin\AppData\Local\Temp\ipykernel_14372\135503381.py:93: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered = filtered_x[(resampled.y >= y_lim_neg + margin ) & (resampled.y <= y_lim_pos - margin )]


In [ ]:
print ('done')

In [ ]:
df.head()